<a href="https://colab.research.google.com/github/gpsilva2003/MPI/blob/main/MPI_LNCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introdução
O ambiente Colab, por padrão, compila e executa códigos em python. Entretanto, vamos utilizá-lo para executar programas em linguagem "C" com uso de uma biblioteca MPI. Devemos ter certeza que os pacotes necessários estão instalados. A principio, os seguintes pacotes serão necessários:

- gcc
- openmpi

Altere o tipo de ambiente de execução para v2-8 TPU. Em caso de dúvidas utilize o apoio do assistente de IA Gemini.


In [4]:
!gcc --version


gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [5]:
!mpicc --version

gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [3]:
!sudo apt install openmpi-doc

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  openmpi-doc
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,072 kB of archives.
After this operation, 4,268 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 openmpi-doc all 4.1.2-2ubuntu1 [1,072 kB]
Fetched 1,072 kB in 0s (4,686 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package openmpi-doc.
(Reading database ... 123632 files 

Em seguida, vamos importar os arquivos a serem utilizados no curso do repositório https://github.com/Programacao-Paralela-e-Distribuida/MPI . Depois organizar os arquivos adequadamente para nossos exemplos.

In [2]:
!git clone https://github.com/Programacao-Paralela-e-Distribuida/MPI

Cloning into 'MPI'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 126 (delta 66), reused 69 (delta 38), pack-reused 0 (from 0)
Receiving objects: 100% (126/126), 3.49 MiB | 13.94 MiB/s, done.
Resolving deltas: 100% (66/66), done.


In [3]:
!mv MPI/* .
!rm -rf MPI
!ls -l

total 36
drwxr-xr-x 2 root root  4096 Jan 13 15:06 bin
drwxr-xr-x 2 root root  4096 Jan 13 15:06 docs
-rw-r--r-- 1 root root   547 Jan 13 15:06 Makefile
-rw-r--r-- 1 root root 13256 Jan 13 15:06 README.md
drwxr-xr-x 1 root root  4096 Jan  9 14:19 sample_data
drwxr-xr-x 2 root root  4096 Jan 13 15:06 src


# Exemplo simples

A seguir apresentamos um exemplo simples de programa MPI. Para sua execução o comando a ser utilizado é o *mpicc* que faz a ligação automática com os arquivos de cabeçalho e bibliotecas necessários para a compilação.

In [4]:
!cat ./src/mpi_simples.c


#include <stdio.h>
#include <string.h>
#include "mpi.h"

int main(int argc, char *argv[]) { /* mpi_simples.c  */
int meu_ranque, num_procs;
int origem, destino, etiq = 0;
char mensagem[200];
MPI_Status estado;
    /* Chamada inicial para o MPI */
    MPI_Init(&argc, &argv);
    /* Determina ranque e número de processos em execução */
    MPI_Comm_rank(MPI_COMM_WORLD, &meu_ranque);
    MPI_Comm_size(MPI_COMM_WORLD, &num_procs);
    /* Todos os processos com ranque diferente de 0 enviam uma mensagem */
    if (meu_ranque != 0) {
        sprintf(mensagem, "Processo %d está vivo!", meu_ranque);
        destino = 0;
        MPI_Send(mensagem, strlen(mensagem)+1, MPI_CHAR, destino, etiq, MPI_COMM_WORLD);
    } 
    /* Processo com ranque 0 recebe num_procs-1 mensagens */
    else {
        for (origem = 1; origem < num_procs; origem++) {
            MPI_Recv(mensagem, 200, MPI_CHAR, origem, etiq, MPI_COMM_WORLD, &estado);
    /* Imprime as mensagens recebidas */
            printf("%s\n",men

In [7]:
!mpicc -o bin/mpi_simples src/mpi_simples.c
!mpirun --allow-run-as-root -np 4 ./bin/mpi_simples

Processo 1 está vivo!
Processo 2 está vivo!
Processo 3 está vivo!
